In [789]:
import math
import random
import 

SyntaxError: invalid syntax (<ipython-input-789-e35c159e626c>, line 3)

In [790]:
def exponencial(lde,t):
    return -((1/lde)*math.log(random.random()))

In [791]:
def poisson(ldpp,t):
    return t-((1/ldpp)*math.log(random.random()))

In [792]:
class task:
    def __init__(self, ta):
        self.t_llegada = ta
        self.t_atendido = float('Inf')
        self.t_salida = float('Inf')
    
    #devuelve el tiempo total del task
    def get_tiempoT(self):
        return (self.t_salida - self.t_llegada)
    
    #devuelve el tiempo en servidor
    def get_tiempoEnServ(self):
        return (self.t_salida - self.t_atendido)
    
    #devuelve tiempo de salida

In [793]:
class server:
    def __init__(self):
        self.bussy = False
        self.task_running = False
        self.tasks = [] #processed tasks
        self.Tp = 0
    
    def assign_task(self, taske):
        self.bussy = True
        self.task_running = taske
        
    def drop_task(self):
        self.bussy = False
        self.tasks.append(self.task_running)
        self.task_running = False
        

In [794]:
class SystemSim:
    def __init__(self, T, ldp, lde, servidores):
        #flags
        self.serv_working = True #cambia hasta el caso 4
        #parametros cambian
        self.T = T
        self.Tp_general = 0
        self.ldp = ldp
        self.lde = lde
        
        #parametros constantes
        self.t = 0
        self.Na = 0
        self.Nd = 0
        self.ta = poisson(ldp,self.t)
        
        #listas
        self.cola_eventos = []
        self.tiempos_salida = []
        self.servidores = []
        
        #inicializo objetos servidor
        for i in range(servidores):
            self.servidores.append(server())
            self.tiempos_salida.append(float('Inf'))
            
    #Verifica si aun hay tiempo de ejecucion
    def get_tiempoDisp(self):
        if(self.t < self.T and self.serv_working):
            return True
        else:
            return False
    
    #agrega un nuevo objeto a la cola
    def add_task(self,ta):
        servidor_pos = self.get_free()
        self.cola_eventos.append(task(ta)) #### se mete task a la cola
        #existe un servidor desocupado
        if(servidor_pos > -1):
            self.assign_task(ta,servidor_pos)
    
    def assign_task(self,tx,index):
        self.servidores[index].assign_task(self.cola_eventos[0]) #se agrega al servidor seleccionado
        self.servidores[index].task_running.t_atendido = tx #se asigna el tiempo de atendido
        self.servidores[index].task_running.t_salida = self.t + exponencial(self.lde,self.t) #se asigna tiempo de salida
        self.cola_eventos.pop(0) #task atendiendo
        
    #identifica que servidor esta libre
    def get_free(self):
        #devuelve -1 si no hay libres
        for i in range(len(self.servidores)):
            if(not self.servidores[i].bussy):
                return i
        return -1
    
    #devuelve el tiempo de salida mas pequeño
    def get_td(self):
        td_min = float('Inf')
        td_servidor = -1
        for i in range(len(self.servidores)):
            if((self.servidores[i].task_running != False) and (self.servidores[i].task_running.t_salida < td_min)):
                td_min = self.servidores[i].task_running.t_salida
                td_servidor = i
        return (td_min,td_servidor)
        
    #debug block
    def debug(self):
        print("t: ", self.t, "\tta: ", self.ta, "\ttd: ", self.get_td()[0])
        print("Servidores: ",end='')
        for i in self.servidores:
            print("\t",end='')
            print(i.bussy,end='')
        print("\ttasks_cola: ", end='')
        print(len(self.cola_eventos),end='')
        print("")
        

In [795]:
def caso1(world):
    #print("CASO 1",end='')
    if(world.ta <= world.T):
        #world.debug()
        world.t = world.ta
        world.Na += 1
        world.ta = poisson(world.ldp,world.t)
        #print("\tNew_ta: ",world.ta)
        world.add_task(world.t)

In [796]:
def caso2(world):
    #print("CASO 2")
    #world.debug()
    world.t = world.get_td()[0]
    servidor = world.get_td()[1] #es un indice
    world.Nd += 1
    #world.servidores[servidor].bussy = False
    world.servidores[servidor].drop_task()
    
    if(not world.get_tiempoDisp()):
        world.servidores[servidor].Tp += max(world.t - world.T, 0)
        
    if(len(world.cola_eventos)>0):
        world.assign_task(world.t,servidor)
        
        

In [797]:
def caso4(world):
    #print("CASO 4")
    world.serv_working = False

In [798]:
def selector(world):
    if((world.ta < world.get_td()[0]) and (world.ta < world.T)):
        # El siguiente evento es una llegada de una solicitud al 
        # sistema y aún no es la hora de cierre. 
        caso1(world)
    elif(world.get_td()[0] < world.ta):
        # El siguiente evento es una salida de una solicitud del
        # sistema
        caso2(world)
    else:
        caso4(world)

In [799]:
def simulador(T,ldp,lde,servidores):
    #se crea el sistema con todos los parametros
    world = SystemSim(T,ldp,lde,servidores)
    
    while(world.get_tiempoDisp()):
        #print("")
        #world.debug()
        selector(world)
    return world
    

In [800]:
def get_data(world):
    #info del sistema
    print("INFORMACIÓN DEL SISTEMA: ")
    print("\t\tNa: ", world.Na,"\tNd: ", world.Nd)
    
    #solicitudes de cada servidor
    print("\nSOLICITUDES:")
    for i in range(len(world.servidores)):
        print("\t\tServidor ",i+1,"\t--> ",len(world.servidores[i].tasks))
        
    #tiempo ocupado de cada servidor
    tiempos_servidores = [] #lo almaceno para calcular IDDLE
    print("\nTIEMPO OCUPADOS:")
    for i in range(len(world.servidores)):
        #servidor X:
        tiempo_ocupado = 0
        for j in world.servidores[i].tasks:
            tiempo_ocupado += (j.t_salida - j.t_atendido)
        tiempos_servidores.append(tiempo_ocupado)
        print("\t\tServidor ",i+1,": ",tiempo_ocupado)
    
    #tiempo IDDLE
    print("\nTIEMPO DESOCUPADOS:")
    for i in range(len(tiempos_servidores)):
        print("\t\tServidor ",i+1,": ",(world.T - tiempos_servidores[i]))

    #Tiempo total de solicitudes en cola
    print("\nTIEMPO DE SOLICITUDES EN COLA:")
    tiempo_en_cola_total = 0
    for i in range(len(world.servidores)):
        #servidor X:
        for j in world.servidores[i].tasks:
            tiempo_en_cola_total += (j.t_atendido - j.t_llegada)
    print("\t\tTiempo total: ", tiempo_en_cola_total)
    
    #Promedio cuantas tasks en cola/seg
    contadores = []
    contador = 0
    print("\nPROMEDIO DE SOLICITUDES EN COLA POR SEGUNDO:")
    ###Esta parte se tarda demasiado y no di como evaluarla de otra manera
    #se analizan los datos por intervalos de 1 segundo
    #for i in range(world.T+1):
        #contador = 0
        #for j in world.servidores:
            #for k in j.tasks:
                #if((k.t_llegada < i) and (k.t_llegada > i-1)):
                    #está en el intervalo evaluado, verifico que esperó en cola
                    #if((k.t_atendido - k.t_llegada)>0):
                        #estuvo en cola un tiempo en este intervalo
                        #contador +=1
                #elif((k.t_llegada < i) and (k.t_llegada < i-1)):
                    #estuvo esperando un tiempo en este intervalo
                    #if(k.t_atendido > i-1):
                        #contador +=1
                    #else (no es parte del intervalo)
        #contadores.append(contador)
    
    print("\t\tPromedio de solicitudes en cola por segundo: ", contador/world.T)
    
    #tiempo de salida de la última solicitud
    print("\nTIEMPO DE SALIDA DE LA ÚLTIMA SOLICITUD: ")
    print("\t\tTiempo final: ",world.t)
    for i in range(len(world.servidores)):
        print("\t\tExtra Servidor ", i,": ",world.servidores[i].Tp)

In [801]:
simulacion = simulador(3600,2400/60,100,2)
print("---> Finished: SIMULACION 1 <---")
get_data(simulacion)

---> Finished: SIMULACION 1 <---
INFORMACIÓN DEL SISTEMA: 
		Na:  144082 	Nd:  144082

SOLICITUDES:
		Servidor  1 	-->  106403
		Servidor  2 	-->  37679

TIEMPO OCUPADOS:
		Servidor  1 :  1059.734804278147
		Servidor  2 :  376.4748697976295

TIEMPO DESOCUPADOS:
		Servidor  1 :  2540.265195721853
		Servidor  2 :  3223.5251302023707

TIEMPO DE SOLICITUDES EN COLA:
		Tiempo total:  57.92714013510542

PROMEDIO DE SOLICITUDES EN COLA POR SEGUNDO:
		Promedio de solicitudes en cola por segundo:  0.0

TIEMPO DE SALIDA DE LA ÚLTIMA SOLICITUD: 
		Tiempo final:  3599.941905525269
		Extra Servidor  0 :  0
		Extra Servidor  1 :  0
